# 🚀 Image Upscaler AI

**Transform low-resolution images into high-quality visuals using state-of-the-art AI models.**

## 📖 About

Image Upscaler AI is a powerful, easy-to-use image enhancement tool that runs entirely in Google Colab. Leveraging Real-ESRGAN for super-resolution and GFPGAN for face restoration, this project enables anyone to upscale and enhance images up to 4x resolution with professional-grade results—no technical expertise required.

## ✨ Features

- 🔍 **2x & 4x Upscaling** - Enhance image resolution with Real-ESRGAN
- 🎭 **Face Enhancement** - Restore and improve facial details with GFPGAN
- 🖼️ **Interactive UI** - User-friendly Gradio web interface
- 📊 **Comparison View** - Side-by-side before/after visualization
- 🗂️ **Batch Processing** - Process multiple images at once
- ⚡ **GPU Accelerated** - Fast processing with CUDA support
- 💾 **High Quality Output** - Preserves image quality and details

## 🚀 Quick Start

1. Open the notebook in Google Colab
2. Run all cells in order
3. Upload your image in the Gradio interface
4. Select upscale factor (2x or 4x)
5. Click "Upscale Image" and download results!

Perfect for photographers, designers, content creators, and anyone looking to enhance their images effortlessly.



In [1]:
# Install all required packages with compatible versions
print("📦 Installing dependencies... This will take 2-3 minutes.")

# Install PyTorch with CUDA support
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install Real-ESRGAN with all dependencies
!pip install -q realesrgan

# Install GFPGAN for face enhancement
!pip install -q gfpgan

# Install Gradio and utilities
!pip install -q gradio opencv-python-headless Pillow numpy

# Install additional required packages
!pip install -q basicsr facexlib

print("\n" + "="*60)
print("✅ ALL PACKAGES INSTALLED SUCCESSFULLY!")
print("="*60)
print("\n🔄 IMPORTANT: RESTART RUNTIME NOW!")
print("   Go to: Runtime → Restart runtime")
print("\n   After restart, run cells 3 onwards (skip this cell)")
print("="*60)

📦 Installing dependencies... This will take 2-3 minutes.

✅ ALL PACKAGES INSTALLED SUCCESSFULLY!

🔄 IMPORTANT: RESTART RUNTIME NOW!
   Go to: Runtime → Restart runtime

   After restart, run cells 3 onwards (skip this cell)


In [2]:
# Download pre-trained models
import os
import urllib.request

print("📥 Downloading AI models...")

model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)

models = {
    'RealESRGAN_x4plus.pth': 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth',
    'RealESRGAN_x2plus.pth': 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
    'GFPGANv1.3.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth'
}

for model_name, model_url in models.items():
    model_path = os.path.join(model_dir, model_name)
    if not os.path.exists(model_path):
        print(f"📥 Downloading {model_name}... (~170MB)")
        urllib.request.urlretrieve(model_url, model_path)
        print(f"✅ {model_name} downloaded!")
    else:
        print(f"✅ {model_name} already exists!")

print("\n✅ All models ready!")


📥 Downloading AI models...
✅ RealESRGAN_x4plus.pth already exists!
✅ RealESRGAN_x2plus.pth already exists!
✅ GFPGANv1.3.pth already exists!

✅ All models ready!


In [3]:
# Install compatible torchvision version
# This addresses a known incompatibility issue where basicsr (a dependency of realesrgan)
# expects an older version of torchvision.
# NOTE: Attempting to install torchvision==0.15.2 failed as it's not available for current Python/Torch setup.
# We will use a monkey-patch workaround instead.
# !pip install -q torchvision==0.15.2 # This line is now commented out as it failed.

import sys
import importlib

# --- Start of Monkey Patch for torchvision compatibility ---
# The basicsr library expects 'torchvision.transforms.functional_tensor' which is removed in newer torchvision versions.
# We will create a proxy module to redirect the import.
try:
    # Try to import the new functional module
    from torchvision.transforms import functional as _functional_new

    # Create a simple object to act as the 'functional_tensor' module
    class FunctionalTensorProxyModule:
        def __getattr__(self, name):
            if hasattr(_functional_new, name):
                return getattr(_functional_new, name)
            raise AttributeError(f"module 'torchvision.transforms.functional_tensor' has no attribute '{name}'")

    # Only patch if 'functional_tensor' is not already in sys.modules
    if 'torchvision.transforms.functional_tensor' not in sys.modules:
        sys.modules['torchvision.transforms.functional_tensor'] = FunctionalTensorProxyModule()
        print("✅ Monkey-patched torchvision.transforms.functional_tensor for basicsr compatibility.")
    else:
        print("ℹ️ torchvision.transforms.functional_tensor already exists, no monkey-patch needed.")

except ImportError:
    print("⚠️ Could not import torchvision.transforms.functional. Monkey-patch failed. Ensure torchvision is installed.")
# --- End of Monkey Patch ---


# Import necessary libraries
import cv2
import numpy as np
import torch
from PIL import Image
import gradio as gr
import os

# Import Real-ESRGAN
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

# Import GFPGAN
from gfpgan import GFPGANer

print("="*60)
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ Running on CPU (will be slower)")
print("="*60)


✅ Monkey-patched torchvision.transforms.functional_tensor for basicsr compatibility.
✅ PyTorch version: 2.9.0+cu126
✅ CUDA available: True
✅ GPU: Tesla T4
✅ GPU Memory: 15.83 GB


In [4]:
# Initialize Real-ESRGAN models
def load_realesrgan_model(scale=4):
    """Load Real-ESRGAN model with specified scale"""
    model_name = f'RealESRGAN_x{scale}plus'
    model_path = f'models/RealESRGAN_x{scale}plus.pth'

    # Define model architecture
    if scale == 4:
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
    else:
        model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)

    # Initialize upsampler
    upsampler = RealESRGANer(
        scale=scale,
        model_path=model_path,
        model=model,
        tile=0,
        tile_pad=10,
        pre_pad=0,
        half=True if torch.cuda.is_available() else False,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )

    return upsampler

# Initialize GFPGAN for face enhancement
def load_gfpgan_model():
    """Load GFPGAN model for face restoration"""
    gfpgan_path = 'models/GFPGANv1.3.pth'

    bg_upsampler = load_realesrgan_model(scale=2)

    face_enhancer = GFPGANer(
        model_path=gfpgan_path,
        upscale=2,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=bg_upsampler,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )

    return face_enhancer

print("✅ Model loading functions initialized!")


✅ Model loading functions initialized!


In [5]:
# Core image processing functions
def upscale_image(image, scale=4, enhance_face=False):
    """
    Upscale image using Real-ESRGAN with optional face enhancement

    Args:
        image: PIL Image or numpy array
        scale: Upscaling factor (2 or 4)
        enhance_face: Whether to apply GFPGAN face enhancement

    Returns:
        Upscaled image as numpy array
    """
    try:
        # Convert PIL to numpy if needed
        if isinstance(image, Image.Image):
            image = np.array(image)

        # Convert RGB to BGR for OpenCV
        img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if enhance_face:
            print("🎭 Applying GFPGAN face enhancement...")
            face_enhancer = load_gfpgan_model()
            _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
        else:
            print(f"🖼️ Upscaling with Real-ESRGAN x{scale}...")
            upsampler = load_realesrgan_model(scale=scale)
            output, _ = upsampler.enhance(img, outscale=scale)

        # Convert BGR back to RGB
        output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

        print("✅ Image upscaled successfully!")
        return output

    except Exception as e:
        print(f"❌ Error during upscaling: {str(e)}")
        return image

def create_comparison(original, upscaled):
    """Create side-by-side comparison of original and upscaled images"""
    # Resize original to match upscaled height for comparison
    h_up, w_up = upscaled.shape[:2]
    h_orig, w_orig = original.shape[:2]

    # Calculate new dimensions maintaining aspect ratio
    scale_factor = h_up / h_orig
    new_w = int(w_orig * scale_factor)

    original_resized = cv2.resize(original, (new_w, h_up), interpolation=cv2.INTER_CUBIC)

    # Concatenate horizontally
    comparison = np.hstack([original_resized, upscaled])

    return comparison

print("✅ Core processing functions ready!")


✅ Core processing functions ready!


In [6]:
# Gradio interface processing function
def process_image(image, upscale_factor, face_enhance, show_comparison):
    """
    Process image through Gradio interface

    Args:
        image: Input image from Gradio
        upscale_factor: Upscaling factor (2x or 4x)
        face_enhance: Whether to use GFPGAN
        show_comparison: Whether to show side-by-side comparison

    Returns:
        Processed image(s)
    """
    if image is None:
        return None, "⚠️ Please upload an image first!"

    try:
        # Get scale from dropdown
        scale = int(upscale_factor[0])  # Extract number from "2x" or "4x"

        # Convert to numpy array
        original = np.array(image)

        # Process image
        upscaled = upscale_image(original, scale=scale, enhance_face=face_enhance)

        # Get image info
        orig_h, orig_w = original.shape[:2]
        up_h, up_w = upscaled.shape[:2]

        info = f"""
## 📊 Processing Complete!

**Original Size:** {orig_w} x {orig_h} pixels
**Upscaled Size:** {up_w} x {up_h} pixels
**Scale Factor:** {scale}x
**Face Enhancement:** {'✅ Enabled' if face_enhance else '❌ Disabled'}
**Improvement:** {(up_w * up_h) / (orig_w * orig_h):.1f}x more pixels
"""

        if show_comparison:
            output = create_comparison(original, upscaled)
            info += "\n\n**View Mode:** Side-by-side comparison (Original | Upscaled)"
        else:
            output = upscaled
            info += "\n\n**View Mode:** Upscaled image only"

        return output, info

    except Exception as e:
        error_msg = f"❌ Error: {str(e)}\n\nPlease try again or use a different image."
        return None, error_msg

print("✅ Gradio interface function ready!")


✅ Gradio interface function ready!


In [7]:
# Create Gradio interface
def create_gradio_app():
    """Create and launch Gradio web interface"""

    with gr.Blocks(title="AI Image Upscaler", theme=gr.themes.Soft()) as app:
        gr.Markdown("""
        # 🎨 AI Image Upscaler Pro
        ### Powered by Real-ESRGAN & GFPGAN

        Upload any image and enhance it using state-of-the-art AI models! Perfect for:
        - 📸 Old photos restoration
        - 🖼️ Low-resolution images
        - 👤 Portrait enhancement
        - 🎮 Gaming screenshots
        """)

        with gr.Row():
            with gr.Column(scale=1):
                input_image = gr.Image(type="pil", label="📤 Upload Image")

                gr.Markdown("### ⚙️ Settings")

                upscale_factor = gr.Dropdown(
                    choices=["2x", "4x"],
                    value="4x",
                    label="🔍 Upscale Factor",
                    info="2x = Faster, 4x = Best Quality"
                )

                face_enhance = gr.Checkbox(
                    label="🎭 Enable Face Enhancement (GFPGAN)",
                    value=False,
                    info="Improves facial details and skin texture"
                )

                show_comparison = gr.Checkbox(
                    label="📊 Show Side-by-Side Comparison",
                    value=True,
                    info="View original and upscaled images together"
                )

                process_btn = gr.Button("✨ Upscale Image", variant="primary", size="lg")

                gr.Markdown("""
                ### 💡 Tips:
                - **2x Upscale**: Faster, good for moderate enhancement
                - **4x Upscale**: Best quality, recommended for final output
                - **Face Enhancement**: Enable for portraits and selfies
                - **Comparison View**: Helps see the improvement
                """)

            with gr.Column(scale=1):
                output_image = gr.Image(label="🖼️ Enhanced Result", type="numpy")
                output_info = gr.Markdown("Upload an image and click 'Upscale Image' to start!")

        # Connect button to processing function
        process_btn.click(
            fn=process_image,
            inputs=[input_image, upscale_factor, face_enhance, show_comparison],
            outputs=[output_image, output_info]
        )

        gr.Markdown("""
        ---
        ### 📝 About the Models:
        - **Real-ESRGAN**: Deep learning model trained on millions of images for superior upscaling
        - **GFPGAN**: Specialized GAN for face restoration and enhancement
        - Processing time: 5-30 seconds depending on image size and GPU availability
        """)

    return app

print("✅ Gradio app created successfully!")


✅ Gradio app created successfully!


In [ ]:
# Launch Gradio interface
print("="*60)
print("🚀 Launching AI Image Upscaler...")
print("="*60)
print("⏳ Loading models... (this may take 10-20 seconds)")

app = create_gradio_app()
app.launch(
    share=True,  # Create public shareable link
    debug=True,
    server_name="0.0.0.0",
    server_port=7860,
    show_error=True
)

print("\n" + "="*60)
print("✅ Application is running!")
print("="*60)
print("📱 Use the URL above to access the interface")
print("🔗 Share the 'public URL' with others (valid for 72 hours)")
print("="*60)


🚀 Launching AI Image Upscaler...
⏳ Loading models... (this may take 10-20 seconds)


/tmp/ipython-input-3857613277.py:5: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="AI Image Upscaler", theme=gr.themes.Soft()) as app:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7c9783200137c474e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🖼️ Upscaling with Real-ESRGAN x4...
✅ Image upscaled successfully!


## 🚀 How to Run - 5 Simple Steps

### Step 1: Enable GPU
- Go to **Runtime → Change runtime type**
- Select **T4 GPU** as Hardware accelerator
- Click **Save**

### Step 2: Run All Cells
- Go to **Runtime → Run all**
- Wait 3-5 minutes for installation and setup
- When prompted, click **"Restart Runtime"** (around Cell 2)
- After restart, **Run all cells again** (Runtime → Run all)

### Step 3: Wait for Gradio Link
- Scroll down to Cell 10 output
- Wait for the **public URL** to appear
- Look for: `Running on public URL: https://...`

### Step 4: Open the Interface
- Click on the **public URL** link
- Gradio web interface will open in a new tab

### Step 5: Upscale Your Image
1. **Upload** your image
2. **Select** upscale factor (2x or 4x)
3. **Enable** face enhancement (optional)
4. **Click** "✨ Upscale Image" button
5. **Download** your enhanced result!

---

## ✅ That's it! Your image is now upscaled and enhanced.

**Total time:** ~5 minutes first run, ~30 seconds per image upscaling
